##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Prerequisite: Upload the gesture data folder to the files here. It can be found in the code base

In [ ]:
! rm -r hand_pass_model2

In [ ]:
!unzip -qq gesture_data.zip

In [ ]:
import matplotlib.pyplot as plt
import os

NUM_EXAMPLES = 5
IMAGES_PATH = "dataset" # "rps_data_sample"

# Get the list of labels from the list of folder names.
labels = []
for i in os.listdir(IMAGES_PATH):
  if os.path.isdir(os.path.join(IMAGES_PATH, i)):
    labels.append(i)

# Show the images.
for label in labels:
  label_dir = os.path.join(IMAGES_PATH, label)
  print(label_dir)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  print(example_filenames)
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

dataset/13
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/15
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/16
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/14
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/11
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/12
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']
dataset/None
['963.jpg', '1168.jpg', '939.jpg', '1045.jpg', '1028.jpg']


## Making a New Model



In [ ]:
!pip install -q mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 86.1 MB/s eta 0:00:00


In [ ]:
# Import the necessary modules.
from mediapipe_model_maker import gesture_recognizer

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
gesture_recognizer.ModelOptions

mediapipe_model_maker.python.vision.gesture_recognizer.model_options.GestureRecognizerModelOptions

In [ ]:
# Load the gesture image archive.
data = gesture_recognizer.Dataset.from_folder(
    dirname=IMAGES_PATH,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)

# Split the archive into training, validation and test dataset.
train_data, rest_data = data.split(0.80)
validation_data, test_data = rest_data.split(0.50)

ValueError: No valid hand is detected.

In [ ]:
# Train the model
hparams = gesture_recognizer.HParams(export_dir="hand_pass_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_119 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 re_lu_119 (ReLU)            (None, 128)               0         
                                                                 
 dropout_119 (Dropout)       (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 7)                 903       
 out (Dense)                                                     
                                                          

In [ ]:
# Train the custom model
hparams2 = gesture_recognizer.HParams(export_dir="hand_pass_model2", epochs=10, learning_rate = 0.001, steps_per_epoch=128)
custom_option = gesture_recognizer.ModelOptions(dropout_rate=0.3, layer_widths=[32])
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams2, model_options=custom_option)
model2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_79"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_163 (B  (None, 128)               512       
 atchNormalization)                                              
                                                                 
 re_lu_163 (ReLU)            (None, 128)               0         
                                                                 
 dropout_163 (Dropout)       (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 32)                4128      
 0 (Dense)                                                       
                                                          

Let's see the details of the models

In [ ]:
loss, acc = model2.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

60/60 [==============================] - 0s 2ms/step - loss: 0.1754 - categorical_accuracy: 0.8833
Test loss:0.17535260319709778, Test accuracy:0.8833333253860474


In [ ]:
model.evaluate(test_data, batch_size=1)

60/60 [==============================] - 1s 2ms/step - loss: 0.1736 - categorical_accuracy: 0.8500


[0.1736307293176651, 0.8500000238418579]

In [ ]:
model2.evaluate(test_data, batch_size=1)

60/60 [==============================] - 1s 2ms/step - loss: 0.3998 - categorical_accuracy: 0.8667


[0.39977094531059265, 0.8666666746139526]

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 7)                 903       
 out (Dense)                                                     
                                                             

In [ ]:
# Export the model bundle.
model.export_model()
# Rename the file to be more descriptive.
!mv hand_pass_model/gesture_recognizer.task hand_pass.task

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


In [ ]:
from google.colab import files
files.download("hand_pass.task")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>